# IMPORT

In [1]:
from scipy.io import loadmat
import numpy as np
from sklearn.decomposition import PCA

In [3]:
# Webcam
W = loadmat('surf/webcam.mat')
X_w = W['fts']
Y_w = W['labels']

# Dslr
D = loadmat('surf/dslr.mat')
X_d = D['fts']
Y_d = D['labels']


In [4]:
print(X_w.shape)
print(Y_w.shape)
print(X_d.shape)
print(Y_d.shape)

(295, 800)
(295, 1)
(157, 800)
(157, 1)


# Subspace Alignement

## Without optimal transport

## With optimal transport

In [ ]:
scaler = StandardScaler()


# Entropic regularized optimal transport